Load all libraries

In [354]:
import pandas as pd
import itertools, nltk, string 
import requests, re
from nltk import Tree
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import os

wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer("english")


Read Dataset

In [7]:
res_single_df = pd.read_csv('../restaurant-single-categories.csv')
res_multiple_df = pd.read_csv('../restaurant-multiple-categories.csv')

lap_single_df = pd.read_csv('../laptop-single-categories.csv')
lap_multiple_df = pd.read_csv('../laptop-multiple-categories.csv')

Dataset Exploration

In [8]:
res_single_df.head()

,reviewID,sentenceID,review,category,polarity
0,1004293,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,negative
1,1004293,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE#GENERAL,negative
2,1004293,1004293:2,"They never brought us complimentary noodles, i...",SERVICE#GENERAL,negative
3,1004293,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#QUALITY,negative
4,1004293,1004293:4,"After all that, they complained to me about th...",SERVICE#GENERAL,negative


restaurant categories

In [13]:
res_single_df.category.value_counts()

FOOD#QUALITY                383
RESTAURANT#GENERAL          245
SERVICE#GENERAL             183
AMBIENCE#GENERAL            111
FOOD#STYLE_OPTIONS           51
RESTAURANT#MISCELLANEOUS     50
RESTAURANT#PRICES            24
DRINKS#QUALITY               21
FOOD#PRICES                  17
DRINKS#STYLE_OPTIONS         16
LOCATION#GENERAL             16
DRINKS#PRICES                 3
Name: category, dtype: int64

load lexicon

In [397]:
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))

read_lexicon()

Preprocessing Function

In [398]:
linking_verbs_be = [
    'be',
    'is',
    'are',
    'am',
    'was',
    'were',
    'can be',
    'could be',
    'will be',
    'would be',
    'shall be',
    'should be',
    'may be',
    'might be',
    'must be',
    'has been',
    'have been',
    'had been'
];

linking_verbs_v = [
    'feel',
    'look',
    'smell',
    'sound',
    'taste',
    'act',
    'appear',
    'become',
    'get',
    'grow',
    'prove',
    'remain', 
    'seem',
    'stay',
    'turn'
];

def check_is_noun(pos):
    return re.match('NN.*', pos)

def check_is_verb(pos):
    return re.match('VB.*', pos)

def check_is_adjective(pos):
    return re.match('JJ.*', pos)

def check_is_adverb(pos):
    return re.match('RB.*', pos)

def lemmatize(word, pos):
    tag = wn.NOUN
    if(check_is_noun(pos)):
        tag = wn.NOUN
    elif(check_is_verb(pos)):
        tag = wn.VERB
    elif(check_is_adjective(pos)):
        tag = wn.ADJ
    elif(check_is_adverb(pos)):
        tag = wn.ADV
            
    lemma = wordnet_lemmatizer.lemmatize(word, tag)
    return lemma

def preprocessing(sentence):
    #res = re.sub(' +', ' ', re.sub(r'[^\w\s]','',sentence.replace("'m", "am").replace("n't", "not").replace("'s", ''))).lower()
    res = re.sub(r'[^\w\s]','', sentence.replace("'m", "am").replace("n't", "not").replace("'s", '')).lower()
    #checking for parallel clauses
    #splitted = res.split(', and but')
    res = re.sub(r'\b\d+\b', 'NUM', res)
    return res

def pos_tag(sentence):
    url = "http://localhost:9000"
    request_params = {"annotators": "pos"}
    r = requests.post(url, data=sentence, params=request_params, timeout=120)
    try:
        results = r.json()['sentences'][0]['tokens']
        res = []
        for pos in results:
            res.append((pos['word'], pos['pos']))
        return res
    except Exception as e:
        print(e)
        return []

def get_tregex(text, tregex):
    url = "http://localhost:9000/tregex"
    request_params = {"pattern": tregex}
    r = requests.post(url, data=text, params=request_params, timeout=120)
    try:
        return r.json()['sentences'][0]
    except:
        return []

def sentence_from_tree(s):
    pattern = r'(?<= )[a-zA-Z].*?(?=\))'
    replaced = s.replace('\r\n', '')
    res = ' '.join(re.findall(pattern, replaced))
    return res
        
def sentence_type(clauses):
    IC = 0
    DC = 0
    for clause in clauses:
        if(clause[1] == 'IC'):
            IC += 1
        elif(clause[1] == 'DC'):
            DC += 1

    if IC == 1 and DC == 0:
        return 'simple_sentence'
    elif IC >= 2 and DC == 0:
        return 'compound_sentence'
    elif IC ==1 and DC >= 1:
        return 'complex_sentence'
    elif IC > 1 and DC >= 1:
        return 'compound_complex_sentence'
    else:
        return 'phrase'
    

phrase extraction

In [399]:
def get_phrases(sentence):
    np_tree = get_tregex(sentence, 'NP < NN | < NNS')
    np_temp = []
    
    if np_tree:
        for x in range(0, len(np_tree)):
            phrase = " ".join(Tree.fromstring(np_tree[str(x)]['match']).leaves())
            if not any(phrase in s for s in np_temp):
                np_temp.append(phrase)
     
    advp_tree = get_tregex(sentence, 'ADVP')
    advp_temp = []
    
    if advp_tree:
         for x in range(0, len(advp_tree)):
            phrase = " ".join(Tree.fromstring(advp_tree[str(x)]['match']).leaves())
            if not any(phrase in s for s in advp_temp):
                advp_temp.append(phrase)
     
        
    adjp_tree = get_tregex(sentence, 'ADJP')
    adjp_temp = []
    
    if adjp_tree:
        for x in range(0, len(adjp_tree)):
            phrase = " ".join(Tree.fromstring(adjp_tree[str(x)]['match']).leaves())
            if not any(phrase in s for s in adjp_temp):
                adjp_temp.append(phrase)
    
    pp_tree = get_tregex(sentence, 'PP')
    pp_temp = []
    
    if pp_tree:
        for x in range(0, len(pp_tree)):
            phrase = " ".join(Tree.fromstring(pp_tree[str(x)]['match']).leaves())
            if not any(phrase in s for s in pp_temp):
                pp_temp.append(phrase)
    
    sent_tagged = pos_tag(sentence)
    chunking = []
    
    finish = False
    index = 0
    concat_word = ''
    concat_pos = ''
    while not finish:
        word_tagged = sent_tagged[index]
        concat_word = (concat_word + ' ' + word_tagged[0]).strip()
        concat_pos = (concat_pos + ' ' + word_tagged[1]).strip()
        
        if not check_is_verb(word_tagged[1]) and not word_tagged[1] == 'MD':
            if concat_word in np_temp:
                chunking.append((concat_word.strip(), 'NP', concat_pos))
                concat_word = ''
                concat_pos = ''
           
            if concat_word in pp_temp:
                chunking.append((concat_word.strip(), 'PP', concat_pos))
                concat_word = ''
                concat_pos = ''
                
            if concat_word in adjp_temp:
                chunking.append((concat_word.strip(), 'ADJP', concat_pos))
                concat_word = ''
                concat_pos = ''
            
            if concat_word in advp_temp:
                chunking.append((concat_word.strip(), 'ADVP', concat_pos))
                concat_word = ''
                concat_pos = ''
                
            if word_tagged[1] == 'PRP' or word_tagged[1] == 'FW' and len(concat_word.split()) == 1:
                chunking.append((concat_word.strip(), word_tagged[1], concat_pos))
                concat_word = ''
                concat_pos = ''
                
        else:
            if len(concat_word.split()) == 1:
                next_word = sent_tagged[index + 1] if index + 1 < len(sent_tagged) else ('.', 'END')
                next_next_word = sent_tagged[index + 2] if index + 2 < len(sent_tagged) else ('.', 'END')
                if (check_is_verb(next_word[1]) and check_is_verb(next_next_word[1])) or (check_is_verb(next_next_word[1]) and next_word[1] == 'TO'):
                    chunking.append( (concat_word + ' ' + next_word[0] + ' ' + next_next_word[0], 'VP', 
                                     concat_pos + " " + next_word[1] + ' ' + next_next_word[1]) )
                    concat_word = ''
                    concat_pos = ''
                    index += 2
                elif check_is_verb(next_word[1]) or next_word[1] == 'RP':
                    chunking.append( (concat_word + ' ' + next_word[0], 'VP', concat_pos + ' ' + next_word[1]))
                    concat_word = ''
                    concat_pos = ''
                    index += 1
                else:
                    chunking.append((concat_word.strip(), 'VP', concat_pos))
                    concat_word = ''
                    concat_pos = ''
        
        index += 1
       
        if index >= len(sent_tagged):
            if concat_word != '' and concat_word != '.':
                fail_words = concat_word.split()
                index = index - len(fail_words)
                index = index + 1 if sent_tagged[index][0].strip() != fail_words[0].strip() else index
                chunking.append((sent_tagged[index][0], sent_tagged[index][1], sent_tagged[index][1]))
                concat_word = ''
                concat_pos = ''
                index += 1
            
            if index >= len(sent_tagged):
                finish = True 

    return chunking

get clause function

In [400]:
def get_clauses(sentence):
    temp = []
    clauses = []
    
    res_all_clauses = get_tregex(sentence, 'S < (NP $ VP)') 
    res_sbar_clause = get_tregex(sentence, 'SBAR < S')
    #filter clauses with dependency clauses
    for x in range(0, len(res_all_clauses)):
        s = sentence_from_tree(res_all_clauses[str(x)]['match'])
        ic = True    
        for y in range(0, len(res_sbar_clause)):
            sbar = sentence_from_tree(res_sbar_clause[str(y)]['match'])            
            if sbar in s and sbar != s:
                s = s.replace(sbar, '')
                if(len(res_sbar_clause) == 1 and sbar != ''):
                    temp.append([sbar.strip(), 'DC'])
            elif s in sbar and sbar != '':
                ic = False
                temp.append( [sbar.strip(), 'DC'])
        if ic:
            temp.append( [s.strip(), 'IC'] )

    #overwrite sentence that already exist in list
    len_clause = len(temp)
    for x in range(0, len_clause):
        for y in range(x + 1, len_clause):
            temp[x][0] = temp[x][0].replace(temp[y][0], '').strip()
        
        temp[x][0] = re.sub(r"  ", " ", temp[x][0])
        if(temp[x][0] != ''):
            clauses.append( tuple(temp[x]) )
    #sorted by index sentence
    
    if(len(clauses) == 0):
        clauses.append((sentence, 'Phrase'))
    
    return sorted(clauses, key=lambda clause: 999 if sentence.find(clause[0]) == -1 else sentence.find(clause[0]))

aspect extraction function

In [403]:
 def aspect_extraction(r):
    clauses = get_clauses(r)
    stype = sentence_type(clauses)
    candidate_aspect_per_sentence = []
    candidate_opinion_per_sentence = []
    for c,t in clauses:
        candidate_aspect_per_clause = []
        candidate_opinion_per_clause = []
            
        if stype == 'phrase':
            c_tagged = pos_tag(c)
            
            #Find all noun and append to aspect term
            #for opinion find adjective, verb, and adverb and append to opinion term
            for word, pos in c_tagged:
                if check_is_noun(pos):
                    candidate_aspect_per_clause.append(lemmatize(word, pos))
                elif (check_is_verb(pos) or check_is_adverb(pos) or check_is_adjective(pos)) and (word in positive_lexicon or word in negative_lexicon):
                    candidate_opinion_per_clause.append(lemmatize(word, pos))
        else:
            phrases = get_phrases(c)
                
            is_finish = False;
            index_word = 0
            while not is_finish:
                phrase = phrases[index_word]
                next_phrase = phrases[index_word + 1] if (index_word + 1) != len(phrases) else ('.', 'END', '.')
                #checking verb
                   
                if phrase[1] == 'VP':
                    #aspect always in independet clause:
                    if t == 'IC':
                        if phrase[0] in linking_verbs_be or (phrase[0] in linking_verbs_v and next_phrase[1] != 'NP'):
                            print(phrase, 'linking verb', c, phrases)

                            #linking verb condition
                            #find aspect in subject
                            for i in range(0, index_word):
                                p = phrases[i]
                                if p[1] == 'NP':
                                    if p[1] not in stopWords:
                                        candidate_aspect_per_clause.append(p[0])
                        else:
                            #action verb
                            print(phrase, 'action verb', c, phrases)

                            #checking verb is opinion or not
                            if phrase[0] not in stopWords and (phrase[0] in positive_lexicon or phrase[0] in negative_lexicon):
                                candidate_opinion_per_clause.append(lemmatize(phrase[0], 'VB'))

                            #find aspect in object
                            for i in range(index_word+1, len(phrases)):
                                p = phrases[i]
                                if p[1] == 'NP':
                                    if p[1] not in stopWords:
                                        candidate_aspect_per_clause.append(p[0])
                   
                            #if subject preposition find aspect in preposition
                            if len(candidate_aspect) == 0:
                                #find in pp after verb
                                for i in range(index_word+1, len(phrases)):
                                    p = phrases[i]
                                    words = nltk.word_tokenize(p[0])
                                    word_taggeds = nltk.word_tokenize(p[2])

                                    if p[1] == 'PP':
                                        for i,w in enumerate(words):
                                            if w not in stopWords and check_is_noun(word_taggeds[i]):
                                                candidate_aspect_per_clause.append(lemmatize(w, word_taggeds[i]))

                                #find in pp before verb
                                for i in range(0, index_word):
                                    p = phrases[i]
                                    words = nltk.word_tokenize(p[0])
                                    word_taggeds = nltk.word_tokenize(p[2])

                                    if p[1] == 'PP':
                                        for i,w in enumerate(words):
                                            if w not in stopWords and check_is_noun(word_taggeds[i]):
                                                candidate_aspect_per_clause.append(lemmatize(w, word_taggeds[i]))

                    #find opinion both in IC and DC
                    for i in range(index_word+1, len(phrases)):
                        p = phrases[i]
                        words = nltk.word_tokenize(p[0])
                        word_taggeds = nltk.word_tokenize(p[2])

                        #check opinion in adjective
                        if p[1] == 'ADJP' or p[1] == 'JJ':
                            for i,w in enumerate(words):
                                if w not in stopWords:
                                    candidate_opinion_per_clause.append(lemmatize(w, word_taggeds[i]))          
                        #check opinion in adverb
                        elif p[1] == 'ADVP' or p[1] == 'RB':
                            for i,w in enumerate(words):
                                if w not in stopWords and (w in positive_lexicon or w in negative_lexicon):
                                    candidate_opinion_per_clause.append(lemmatize(w, word_taggeds[i])) 
                        elif p[1] == 'PP':
                            for i,w in enumerate(words):
                                if w not in stopWords and (w in positive_lexicon or w in negative_lexicon):
                                    candidate_opinion_per_clause.append(lemmatize(w, word_taggeds[i]))
                        elif p[1] == 'VP':
                            for i,w in enumerate(words):
                                if w not in stopWords and (w in positive_lexicon or w in negative_lexicon):
                                    candidate_opinion_per_clause.append(lemmatize(w, word_taggeds[i]))
                        elif p[1] == 'NP':
                            for i,w in enumerate(words):
                                if w not in stopWords and (w in positive_lexicon or w in negative_lexicon):
                                    candidate_opinion_per_clause.append(lemmatize(w, word_taggeds[i]))
                                elif w not in stopWords and check_is_noun(word_taggeds[i]):
                                    candidate_aspect_per_clause.append(lemmatize(w, word_taggeds[i]))
                        
                    is_finish = True
                else:
                    index_word += 1
                    if index_word >= len(phrases):
                        is_finish = True
        if len(candidate_aspect_per_clause) > 0 or len(candidate_opinion_per_clause) > 0:
            candidate_aspect_per_sentence.append(candidate_aspect_per_clause)
            candidate_opinion_per_sentence.append(candidate_opinion_per_clause)
            
    return candidate_aspect_per_sentence, candidate_opinion_per_sentence
   

Sentence Preprocessing

In [161]:
#preprocessed sentence and append to new coloum df
arr = []
for r in res_single_df['review']:
    preprocessed_sent = preprocessing(r)
  
    arr.append(preprocessed_sent)
    
preprocess_sent_series = pd.Series(arr)
res_single_df['preprocessed_sentence'] = preprocess_sent_series

aspect and opinion extraction using grammartical rule

In [383]:
aspect_term = []
opinion_term = []

for r in res_single_df.head(100)['preprocessed_sentence']:
    candidate_aspect_per_sentence, candidate_opinion_per_sentence = aspect_extraction(r);
    aspect_term.append(candidate_aspect_per_sentence)
    opinion_term.append(candidate_opinion_per_sentence)

('were imposing', 'VP', 'VBD VBG') action verb like we were imposing on them [('like we', 'PRP', 'IN PRP'), ('were imposing', 'VP', 'VBD VBG'), ('on them', 'PP', 'IN PRP'), ('', 'PRP', '')]
('arrived', 'VP', 'VBD') action verb we arrived [('we', 'PRP', 'PRP'), ('arrived', 'VP', 'VBD')]
('was', 'VP', 'VBD') linking verb at noon the place was empty and the staff acted and they were very rude [('at noon', 'PP', 'IN NN'), ('the place', 'NP', 'DT NN'), ('was', 'VP', 'VBD'), ('empty', 'ADJP', 'JJ'), ('and the staff acted and they', 'PRP', 'CC DT NN VBD CC PRP'), ('were', 'VP', 'VBD'), ('very rude', 'ADJP', 'RB JJ')]
('were imposing', 'VP', 'VBD VBG') action verb at noon the place was empty and the staff acted like we were imposing on them and they were very rude [('at noon the place was empty and the staff acted like we', 'PRP', 'IN NN DT NN VBD JJ CC DT NN VBD IN PRP'), ('were imposing', 'VP', 'VBD VBG'), ('on them', 'PP', 'IN PRP'), ('', 'PRP', ''), ('and they', 'PRP', 'CC PRP'), ('were', 

('was', 'VP', 'VBD') linking verb their sake list was extensive [('their sake list', 'NP', 'PRP$ NN NN'), ('was', 'VP', 'VBD'), ('extensive', 'ADJP', 'JJ')]
('were looking', 'VP', 'VBD VBG') action verb but we were looking for purple haze which wasnot listed but made for us upon request [('but we', 'PRP', 'CC PRP'), ('were looking', 'VP', 'VBD VBG'), ('for purple haze which wasnot listed but made for us', 'PRP', 'IN JJ NN WDT VBP VBN CC VBN IN PRP'), ('upon', 'IN', 'IN'), ('request', 'NP', 'NN')]
('was', 'VP', 'VBD') linking verb the spicy tuna roll was unusually good [('the spicy tuna roll', 'NP', 'DT NN NN NN'), ('was', 'VP', 'VBD'), ('unusually good', 'ADJP', 'RB JJ')]
('was', 'VP', 'VBD') linking verb the rock shrimp tempura was awesome great appetizer to share [('the rock shrimp tempura', 'NP', 'DT NN NN NN'), ('was', 'VP', 'VBD'), ('awesome great appetizer', 'NP', 'JJ JJ NN'), ('to share', 'PP', 'TO NN')]
('went', 'VP', 'VBD') action verb we went around NUM on a friday and [('we'

('have', 'VP', 'VBP') action verb you have yourself the beginning of a great evening [('you', 'PRP', 'PRP'), ('have', 'VP', 'VBP'), ('yourself', 'PRP', 'PRP'), ('the beginning', 'NP', 'DT NN'), ('of a great evening', 'PP', 'IN DT JJ NN')]
('was', 'VP', 'VBD') linking verb the lava cake dessert was incredible [('the lava cake dessert', 'NP', 'DT NN NN NN'), ('was', 'VP', 'VBD'), ('incredible', 'ADJP', 'JJ')]
('recommend', 'VP', 'VB') action verb i recommend it [('i', 'ADVP', 'LS'), ('recommend', 'VP', 'VB'), ('it', 'PRP', 'PRP')]
('is', 'VP', 'VBZ') linking verb this tiny restaurant is as cozy [('this tiny restaurant', 'NP', 'DT JJ NN'), ('is', 'VP', 'VBZ'), ('as cozy', 'ADJP', 'IN JJ')]
('gets', 'VP', 'VBZ') action verb as it gets with that certain parisian flair [('as it', 'PRP', 'IN PRP'), ('gets', 'VP', 'VBZ'), ('with that certain parisian flair', 'PP', 'IN DT JJ JJ NN')]
('was', 'VP', 'VBD') linking verb the food was average to aboveaverage the french onion soup filling yet not ove

In [391]:
aspect_term[25]

[['the food'],
 ['thai fusion stuff', 'bit'],
 ['thai fusion stuff', 'bit'],
 ['every thing']]

In [392]:
opinion_term[25]

[['averagethe'], ['sweet'], ['sweet'], ['sweet']]

In [352]:
get_phrases('the kitchen however is almost always slow')

[('the kitchen', 'NP', 'DT NN'),
 ('however', 'ADVP', 'RB'),
 ('is', 'VP', 'VBZ'),
 ('almost', 'RB', 'RB'),
 ('always', 'RB', 'RB'),
 ('slow', 'VP', 'VB')]

In [359]:
'worth' in positive_lexicon

True

In [393]:
get_clauses('The food is very average...the Thai fusion stuff is a bit too sweet, every thing they serve is too sweet here.')

[('The food is very average', 'IC'),
 ('they serve is too sweet here', 'DC'),
 ('the Thai fusion stuff is a bit too sweet every thing', 'IC')]

In [404]:
candidate_aspect_per_sentence, candidate_opinion_per_sentence = aspect_extraction('The food is very average...the Thai fusion stuff is a bit too sweet, every thing they serve is too sweet here.');
print(candidate_aspect_per_sentence, candidate_opinion_per_sentence)

('is', 'VP', 'VBZ') linking verb The food is very average [('The food', 'NP', 'DT NN'), ('is', 'VP', 'VBZ'), ('very average', 'ADJP', 'RB JJ')]
('serve is', 'VP', 'VBP VBZ') action verb they serve is too sweet here [('they', 'PRP', 'PRP'), ('serve is', 'VP', 'VBP VBZ'), ('too sweet', 'ADJP', 'RB JJ'), ('here', 'ADVP', 'RB')]
('is', 'VP', 'VBZ') linking verb the Thai fusion stuff is a bit too sweet every thing [('the Thai fusion stuff', 'NP', 'DT NNP NN NN'), ('is', 'VP', 'VBZ'), ('a bit', 'NP', 'DT NN'), ('too sweet', 'ADJP', 'RB JJ'), ('every thing', 'NP', 'DT NN')]
[['The food'], [], ['the Thai fusion stuff', 'bit', 'thing']] [['average'], ['sweet'], ['sweet']]


In [405]:
candidate_aspect_per_sentence, candidate_opinion_per_sentence = aspect_extraction('i bought my canon g3 about a month ago and i have to say i am very satisfied');
print(candidate_aspect_per_sentence, candidate_opinion_per_sentence)

('bought', 'VP', 'VBD') action verb i bought my canon g3 about a month ago [('i', 'LS', 'LS'), ('bought', 'VP', 'VBD'), ('my canon g3', 'NP', 'PRP$ NN NN'), ('about a month ago', 'PP', 'IN DT NN RB')]
('have to say', 'VP', 'VBP TO VB') action verb i have to say [('i', 'LS', 'LS'), ('have to say', 'VP', 'VBP TO VB')]
('am', 'VP', 'VBP') linking verb i am very satisfied [('i', 'FW', 'FW'), ('am', 'VP', 'VBP'), ('very satisfied', 'ADJP', 'RB JJ')]
[['my canon g3', 'g3'], []] [[], ['satisfied']]
